In [126]:
import numpy as np
import pandas as pd

In [127]:
data_set = pd.read_csv('Flight_Data.csv')
data_set

,Airline,SourceAirport,DestinationAirport,SourceAirport_City,SourceAirport_Country,SourceAirport_Latitude,SourceAirport_Longitude,SourceAirport_Altitude,DestinationAirport_City,DestinationAirport_Country,DestinationAirport_Latitude,DestinationAirport_Longitude,DestinationAirport_Altitude,Distance,FlyTime,Price
0,Pegasus Airlines,Sabiha Gökçen International Airport,Imam Khomeini International Airport,Istanbul,Turkey,40.898602,29.309200,312,Tehran,Iran,35.416100,51.152199,3305,1998.541333,2.624833,271.489760
1,Turkish Airlines,Atatürk International Airport,Imam Khomeini International Airport,Istanbul,Turkey,40.976898,28.814600,163,Tehran,Iran,35.416100,51.152199,3305,2040.978811,2.882362,300.589499
2,Emirates,Dubai International Airport,Imam Khomeini International Airport,Dubai,United Arab Emirates,25.252800,55.364399,62,Tehran,Iran,35.416100,51.152199,3305,1199.863567,1.536046,210.215879
3,Etihad Airways,Abu Dhabi International Airport,Imam Khomeini International Airport,Abu Dhabi,United Arab Emirates,24.433001,54.651100,88,Tehran,Iran,35.416100,51.152199,3305,1266.681453,2.144199,175.864733
4,Air Arabia,Sharjah International Airport,Imam Khomeini International Airport,Sharjah,United Arab Emirates,25.328600,55.517200,111,Tehran,Iran,35.416100,51.152199,3305,1196.902147,1.772627,174.332843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6831,Kenmore Air,Boeing Field King County International Airport,William R Fairchild International Airport,Seattle,United States,47.529999,-122.302002,21,Port Angeles,United States,48.120201,-123.500000,291,110.930397,0.161072,31.661891
6832,Kenmore Air,Boeing Field King County International Airport,Orcas Island Airport,Seattle,United States,47.529999,-122.302002,21,Eastsound,United States,48.708199,-122.910004,31,138.564899,0.899144,26.201671
6833,Kenmore Air,William R Fairchild International Airport,Boeing Field King County International Airport,Port Angeles,United States,48.120201,-123.500000,291,Seattle,United States,47.529999,-122.302002,21,110.930397,0.751768,99.285196
6834,Kenmore Air,Friday Harbor Airport,Boeing Field King County International Airport,Friday Harbor,United States,48.521999,-123.024002,113,Seattle,United States,47.529999,-122.302002,21,122.677708,1.119010,19.599675


In [141]:
import sys
from queue import PriorityQueue

class Path:
    def __init__(self):
        self.__nodes = []

    @property
    def nodes(self):
        return self.__nodes

    def add_node(self, label: str):
        self.nodes.append(label)

    def __str__(self):
        return f"{self.__nodes}"


class Node:
    def __init__(self, label: str, city : str, country : str, latitude : str, longitude : str, altitude : str):
        self.__label = label
        self.__city = city
        self.__country = country
        self.__latitude = latitude
        self.__longitude = longitude
        self.__altitude = altitude
        self.__edges = []

    @property
    def edges(self):
        return self.__edges

    @property
    def label(self):
        return self.__label

    @property
    def city(self):
        return self.__city

    @property
    def country(self):
        return self.__country

    @property
    def latitude(self):
        return self.__latitude

    @property
    def longitude(self):
        return self.__longitude

    @property
    def altitude(self):
        return self.__altitude

    def add_edge(self, target, distance: float, fly_time : float, price : float):
        edge = Edge(self, target, distance, fly_time, price)
        self.edges.append(edge)

    def __str__(self):
        return f"{self.__label}"


class Edge:

    def __init__(self, from_node: Node, to_node: Node, distance: float, fly_time : float, price : float):
        self.__from_node: Node = from_node
        self.__to_node: Node = to_node
        self.__distance = distance
        self.__fly_time = fly_time
        self.__price = price

    @property
    def from_node(self):
        return self.__from_node

    @property
    def to_node(self):
        return self.__to_node

    @property
    def distance(self):
        return self.__distance

    @property
    def fly_time(self):
        return self.__fly_time

    @property
    def price(self):
        return self.__price

    def __str__(self):
        return f"From : {self.__from_node.city}-{str(self.__from_node)}. {self.__from_node.country}\nTo : {self.__to_node.city}-{str(self.__to_node)}. {self.__to_node.country}\nDistance : {round(self.__distance,2)}\nTime : {round(self.__fly_time, 2)}\nPrice : {round(self.__price, 2)}"


class WeightedGraph:

    def __init__(self):
        self.__nodes = {}  # key: String  value: Node

    @property
    def nodes(self):
        return self.__nodes

    def add_node(self, label: str, city : str, country : str, latitude : str, longitude : str, altitude : str):
        if label not in self.nodes.keys():
            self.nodes[label] = Node(label, city, country, latitude, longitude, altitude)

    def add_edge(self, from_node: str, to_node: str, distance: float, fly_time : float, price : float ):
        origin: Node = self.nodes.get(from_node)
        target: Node = self.nodes.get(to_node)

        if not origin or not target:
            raise Exception("There is not Node for creating Edge")

        origin.add_edge(target, distance, fly_time, price)

    # def find_edge(self, source_node : Node, destination_node : Node):
    #     edges = []
    #     for neighbour in source_node.edges:
    #         if neighbour.to_node == destination_node:
    #             edges.append(neighbour)

    #     return edges.sort(key = lambda x : x.price())[0]

    @classmethod
    def __build_path(cls, previous: dict, to_node: Node) -> Path:
        stack = [to_node]
        previous_node = previous.get(to_node)

        while previous_node:
            stack.append(previous_node[1])
            previous_node: Node = previous.get(previous_node[0])

        p = Path()
        while stack:
            p.add_node(stack.pop())

        return p

    def dijkstra(self, from_node, to_node):
        from_node: Node = self.nodes.get(from_node)
        to_node: Node = self.nodes.get(to_node)

        if not from_node or not to_node:
            raise Exception("There is not Node for finding the shortest path")

        distances = dict()  # key: Node  value:  Integer

        for node in self.nodes.values():
            distances[node] = sys.maxsize

        distances[from_node] = 0

        explored = set()

        frontier = PriorityQueue()
        frontier.put((0, from_node))

        previous = dict()  # key: Node  value: Node

        i = 0
        while not frontier.empty():

            current: Node = frontier.get()[1]
            explored.add(current)

            for edge in current.edges:
                if edge.to_node in explored:
                    continue

                new_distance = distances.get(current) + edge.distance + edge.fly_time + edge.price
                # new_distance = distances.get(current)

                if new_distance < distances.get(edge.to_node):
                    distances[edge.to_node] = new_distance
                    # previous[edge.to_node] = current
                    previous[edge.to_node] = (current, f"Flight #{i}\n"
                                                       f"From: {current.label} - {current.city}, {current.country}\n"
                                                       f"To: {edge.to_node.label} - {edge.to_node.city}, {edge.to_node.country}\n"
                                                       f"Duration: {round(edge.distance, ndigits=2)}\n"
                                                       f"Time: {round(edge.fly_time, ndigits=2)}h"
                                                       f"\nPrice: {round(edge.price, ndigits=2)}$\n"
                                                       f"----------------------------\n")
                    i += 1
                    frontier.put((new_distance, edge.to_node))

        return WeightedGraph.__build_path(previous, to_node)


    def a_star(self, from_node, to_node):

        from_node: Node = self.nodes.get(from_node)
        target_node: Node = self.nodes.get(to_node)

        if not from_node or not target_node:
            raise Exception("There is not Node for finding the shortest path")

        distances = dict()  # key: Node  value:  Integer

        for node in self.nodes.values():
            distances[node] = sys.maxsize

        distances[from_node] = 0

        explored = set()

        frontier = PriorityQueue()
        frontier.put((0, from_node))

        previous = dict()  # key: Node  value: Node


        while not frontier.empty():

            current: Node = frontier.get()[1]
            explored.add(current)

            for edge in current.edges:
                if edge.to_node in explored:
                    continue

                new_distance = distances.get(current) + edge.distance + edge.fly_time + edge.price + self.heuristic(current.latitude, current.longitude, current.altitude, target_node.latitude, target_node.longitude, target_node.altitude, 2)

                if new_distance < distances.get(edge.to_node):
                    distances[edge.to_node] = new_distance
                    previous[edge.to_node] = current
                    frontier.put((new_distance, edge.to_node))
                    if edge.to_node == target_node:   #Goal Check
                        return WeightedGraph.__build_path(previous, target_node)

        return None


    @staticmethod
    def heuristic(source_airport_latitude : float, source_airport_longitude : float, source_airport_altitude : float, destination_airport_latitude : float, destination_airport_longitude : float, destination_airport_altitude : float, k : int):
        return np.power(np.power(source_airport_latitude - destination_airport_latitude, k) +
                        np.power(source_airport_longitude - destination_airport_longitude, k) +
                        np.power(source_airport_altitude - destination_airport_altitude, k), -k)

In [142]:
#Creating The Graph
golabi = WeightedGraph()
for i in range(len(data_set)):
    sample = data_set.iloc[i]
    golabi.add_node(label = sample.SourceAirport , city = sample.SourceAirport_City, country = sample.SourceAirport_Country, latitude = sample.SourceAirport_Latitude, longitude = sample.SourceAirport_Longitude, altitude = sample.SourceAirport_Altitude)
    golabi.add_node(label = sample.DestinationAirport , city = sample.DestinationAirport_City, country = sample.DestinationAirport_Country, latitude = sample.DestinationAirport_Latitude, longitude = sample.DestinationAirport_Longitude, altitude = sample.DestinationAirport_Altitude)
    golabi.add_edge(from_node = sample.SourceAirport, to_node = sample.DestinationAirport, distance = sample.Distance, fly_time = sample.FlyTime, price = sample.Price)

# for sample in golabi.nodes.values():
#     sample: Node
#     for dample in sample.edges:
#         print(dample, end = "\n--------------------------------\n")
        

In [143]:
# source, destination = input().split(" - ")
source, destination = 'Imam Khomeini International Airport', 'Raleigh Durham International Airport'
path = golabi.dijkstra(source, destination)
for p in path.nodes:
    print(p, end="")

Flight #0
From: Imam Khomeini International Airport - Tehran, Iran
To: Atatürk International Airport - Istanbul, Turkey
Duration: 2040.98
Time: 3.15h
Price: 238.35$
----------------------------
Flight #89
From: Atatürk International Airport - Istanbul, Turkey
To: John F Kennedy International Airport - New York, United States
Duration: 8051.74
Time: 10.01h
Price: 836.77$
----------------------------
Flight #134
From: John F Kennedy International Airport - New York, United States
To: Raleigh Durham International Airport - Raleigh-durham, United States
Duration: 686.5
Time: 1.62h
Price: 131.7$
----------------------------
Raleigh Durham International Airport

In [ ]:
# source, destination = input().split(" - ")
# print(golabi.a_star(source, destination))